# Naive Bayes project

In [50]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.6 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 14.2 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.3 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install pandas
!pip install sklearn
!pip install matplotlib



[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [56]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
import pickle
import nltk
from nltk.corpus import stopwords

In [57]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
# Read csv

df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv')

In [14]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [15]:
df_raw.sample(5)

,package_name,review,polarity
182,com.imangi.templerun2,pretty good if i do say so myself oh wait i ...,1
840,com.hamropatro,usefully verry nice app,1
300,com.tencent.mm,wechat 6.3 with wechat out is no longer free ...,0
834,com.hamropatro,well done nicely designed .....this app had c...,1
406,com.facebook.orca,keeps losing connection ever since the update...,0


In [16]:
df_raw['polarity'].value_counts()

0    584
1    307
Name: polarity, dtype: int64

**1. Transform dataframe**

In [17]:
df_transf = df_raw.copy()

In [18]:
# Drop package_name column
# 
df_transf = df_transf.drop('package_name', axis=1)

In [19]:

df_transf['review'] = df_transf['review'].str.strip()
# elimina espacio libre al principio y al final

In [20]:
# column review to lower case
df_transf['review'] = df_transf['review'].str.lower()

In [58]:
stop = stopwords.words('english')

def clean_words(review):
    if review is not None:
        words = review.strip().split()
        new_words = []
        for word in words:
            if word not in stop:new_words.append(word)
        Result = ' '.join(new_words)    
    else:
        Result = None
    return Result


In [59]:
df_transf['review'] = df_transf['review'].apply(clean_words)

In [60]:
df_transf['review'].str.split(expand=True).stack().value_counts()[:60]

app         317
can't       137
new         136
update      136
like        135
fix         133
please      131
good        128
game        121
one         119
use         118
get         112
even        101
great        99
time         93
really       82
would        80
back         78
still        78
love         77
browser      76
make         74
see          69
using        68
work         67
i'm          67
5            67
open         65
want         64
it.          64
google       63
much         61
phone        61
every        61
u            58
option       58
version      57
go           56
way          56
app.         56
used         55
also         55
android      54
download     53
best         52
.            51
give         50
better       50
play         50
send         49
people       48
able         47
message      47
since        46
many         45
keep         45
works        44
i've         44
old          43
find         43
dtype: int64

In [22]:
df = df_transf.copy()

**2. Split data frame**

In [23]:
X = df['review']
y = df['polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=25)


In [24]:
y_train.value_counts()

0    438
1    230
Name: polarity, dtype: int64

In [25]:
y_test.value_counts()

0    146
1     77
Name: polarity, dtype: int64

**3. Pipeline with two pre-processing steps and one modeling step**

In [26]:
vect = CountVectorizer() # vector de conteo

text_vec = vect.fit_transform(X_train)

In [27]:
vect.get_feature_names_out() #da nombres de las columnas

array(['000', '10', '100', ..., 'žŕľ', 'ˇŕ', 'ˇŕľ'], dtype=object)

In [28]:
# text_vec.toarray()[0]

In [29]:
vect_tfidf = TfidfVectorizer()

text_vec_tfidf = vect_tfidf.fit_transform(X_train)

In [30]:

#np.set_printoptions(threshold=sys.maxsize)
#text_vec_tfidf.toarray()[0]


In [31]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer())])
text_norm = text_clf.fit_transform(X_train)

In [32]:
# text_norm.toarray()[0]

In [33]:
clf_1 = MultinomialNB()

clf_1.fit(text_vec, y_train)

MultinomialNB()

In [34]:
clf_2 = MultinomialNB()

clf_2.fit(text_vec_tfidf, y_train)

MultinomialNB()

In [35]:
clf_3 = MultinomialNB()

clf_3.fit(text_norm, y_train)

MultinomialNB()

In [36]:
pred_1 = clf_1.predict(vect.transform(X_test))
pred_2 = clf_2.predict(vect_tfidf.transform(X_test))
pred_3 = clf_3.predict(text_clf.transform(X_test))

print(classification_report(y_test, pred_1))
print(classification_report(y_test, pred_2))
print(classification_report(y_test, pred_3))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       146
           1       0.87      0.61      0.72        77

    accuracy                           0.83       223
   macro avg       0.85      0.78      0.80       223
weighted avg       0.84      0.83      0.83       223

              precision    recall  f1-score   support

           0       0.69      0.99      0.81       146
           1       0.92      0.14      0.25        77

    accuracy                           0.70       223
   macro avg       0.80      0.57      0.53       223
weighted avg       0.77      0.70      0.62       223

              precision    recall  f1-score   support

           0       0.69      0.99      0.81       146
           1       0.92      0.14      0.25        77

    accuracy                           0.70       223
   macro avg       0.80      0.57      0.53       223
weighted avg       0.77      0.70      0.62       223



In [37]:
# Acá todo junto en un pipeline

text_clf_2 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf_2.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

**4. Check results**

In [38]:
y_pred = text_clf_2.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.7664353672100255, 0.6995515695067265, 0.6171890028747934, None)

In [39]:
print('Naive Bayes Train Accuracy = ',metrics.accuracy_score(y_train,text_clf_2.predict(X_train)))
print('Naive Bayes Test Accuracy = ',metrics.accuracy_score(y_test,text_clf_2.predict(X_test)))

Naive Bayes Train Accuracy =  0.7889221556886228
Naive Bayes Test Accuracy =  0.6995515695067265


**5. Randomized search to select hyperparameters**

In [40]:
n_iter_search = 5
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = RandomizedSearchCV(text_clf_2, parameters, n_iter = n_iter_search)
gs_clf.fit(X_train, y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf', MultinomialNB())]),
                   n_iter=5,
                   param_distributions={'clf__alpha': (0.01, 0.001),
                                        'tfidf__use_idf': (True, False),
                                        'vect__ngram_range': [(1, 1), (1, 2)]})

In [41]:
gs_clf.best_params_

{'vect__ngram_range': (1, 2), 'tfidf__use_idf': False, 'clf__alpha': 0.01}

In [42]:
print('Naive Bayes Train Accuracy (grid random search) = ',metrics.accuracy_score(y_train,gs_clf.predict(X_train)))
print('Naive Bayes Test Accuracy (grid random search) = ',metrics.accuracy_score(y_test,gs_clf.predict(X_test)))

Naive Bayes Train Accuracy (grid random search) =  0.9985029940119761
Naive Bayes Test Accuracy (grid random search) =  0.8609865470852018


In [43]:
text_clf_count_vect = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])
text_clf_count_vect.fit(X_train, y_train)


n_iter_search = 5
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'clf__alpha': (1e-2, 1e-3)}
gs_count_vect = RandomizedSearchCV(text_clf_count_vect, parameters, n_iter = n_iter_search)
gs_count_vect.fit(X_train, y_train)

/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 4 is smaller than n_iter=5. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                             ('clf', MultinomialNB())]),
                   n_iter=5,
                   param_distributions={'clf__alpha': (0.01, 0.001),
                                        'vect__ngram_range': [(1, 1), (1, 2)]})

In [44]:
gs_count_vect.best_params_

{'vect__ngram_range': (1, 1), 'clf__alpha': 0.01}

In [45]:
print('Naive Bayes Train Accuracy (grid random search) = ',metrics.accuracy_score(y_train,gs_count_vect.predict(X_train)))
print('Naive Bayes Test Accuracy (grid random search) = ',metrics.accuracy_score(y_test,gs_count_vect.predict(X_test)))

Naive Bayes Train Accuracy (grid random search) =  0.9910179640718563
Naive Bayes Test Accuracy (grid random search) =  0.8251121076233184


In [46]:
y_pred_mejor = gs_clf.predict(X_test)

print(classification_report(y_test, y_pred_mejor))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90       146
           1       0.91      0.66      0.77        77

    accuracy                           0.86       223
   macro avg       0.88      0.81      0.83       223
weighted avg       0.87      0.86      0.85       223



In [47]:
best_model = gs_clf.best_estimator_
best_model

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(alpha=0.01))])

In [48]:
# Save best model

pickle.dump(best_model, open('../models/best_model.pickle', 'wb'))

In [49]:
modelo = pickle.load(open('../models/best_model.pickle', 'rb')) # lo leemos
modelo.predict(X_test) # lo usamos para predecir nueva X_test

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0])